In [ ]:
!pip install psycopg2

In [ ]:
import psycopg2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo -u postgres psql -U postgres -c "CREATE TABLE news(index integer,id integer PRIMARY KEY,domain TEXT,type TEXT,url TEXT,content TEXT,scraped_at DATE,inserted_at DATE,updated_at DATE,title TEXT,authors TEXT,keywords TEXT,meta_keywords TEXT,meta_description TEXT,tags TEXT,summary TEXT,source TEXT);"

In [ ]:
!sudo -u postgres psql -U postgres -c "COPY news FROM '/content/sample_data/1m_data.csv' DELIMITER ',' CSV HEADER ENCODING 'UTF8';"